# Circuit autointerpretability

This stuff just sets up everything we need.

In [49]:
from autointerpretability import *

# Autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
config = yaml.safe_load(open("config.yaml"))
llm_client = AzureOpenAI(
    azure_endpoint=config["base_url"],
    api_key=config["azure_api_key"],
    api_version=config["api_version"],
)

model = HookedTransformer.from_pretrained('gpt2-small')

dataset = load_dataset('Skylion007/openwebtext', split='train', streaming=True)
dataset = dataset.shuffle(seed=42, buffer_size=10_000)
tokenized_owt = tokenize_and_concatenate(dataset, model.tokenizer, max_length=128, streaming=True)
tokenized_owt = tokenized_owt.shuffle(42)
tokenized_owt = tokenized_owt.take(12800 * 2)
owt_tokens = np.stack([x['tokens'] for x in tokenized_owt])
owt_tokens_torch = torch.tensor(owt_tokens)

device = 'cpu'
tl_model, z_saes, transcoders = get_model_encoders(device=device)

/Users/charlesoneill/miniconda3/envs/anu/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded pretrained model gpt2-small into HookedTransformer


/Users/charlesoneill/miniconda3/envs/anu/lib/python3.12/site-packages/datasets/load.py:1486: FutureWarning: The repository for Skylion007/openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Skylion007/openwebtext
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (73252 > 1024). Running this sequence through the model will result in indexing errors


Note you can specify the features you want to examine, in each layer, and just pass in either the relevant ZSAE or MLP transcoder depending on what component you want to look at. The `get_feature_scores` function will handle the differences. Let's have a look at the max-activating examples on Danny's features he wanted to check out (note you can slice `owt_tokens_torch` to run for shorter).

In [7]:
features = [16513, 7861]
sae = z_saes[8]
feature_scores = get_feature_scores(model, sae, owt_tokens_torch, features, batch_size=128)

ZSAE


100%|██████████| 200/200 [04:04<00:00,  1.22s/it]


Our feature scores are a tensor of shape `(batch, feature, seq_pos)`, and so I've got a function to help extract the max-activating examples for each feature. You need to specify the feature index, which is why it's helpful to know from above the features in your list.

In [13]:
feature_scores.shape

(25600, 2, 128)

In [43]:
feature_idx = 0 # corresponding to 16513
example_html, examples_clean_text = display_top_k_activating_examples(model, feature_scores[:, feature_idx, :], owt_tokens_torch, k=15)

In [45]:
top_tokens, top_logits = get_top_k_tokens(model, sae, features[feature_idx], k=20, act_strength=3)

pretty_print_tokens_logits(top_tokens, top_logits)

╒═════════════╤═════════╕
│ Token       │   Logit │
╞═════════════╪═════════╡
│ arth        │  3.6752 │
├─────────────┼─────────┤
│ rers        │  3.4801 │
├─────────────┼─────────┤
│ displayText │  3.3468 │
├─────────────┼─────────┤
│ pool        │  3.3323 │
├─────────────┼─────────┤
│ rovers      │  3.2823 │
├─────────────┼─────────┤
│ qua         │  3.28   │
├─────────────┼─────────┤
│ assian      │  3.2042 │
├─────────────┼─────────┤
│ cember      │  3.1544 │
├─────────────┼─────────┤
│ rer         │  3.1482 │
├─────────────┼─────────┤
│ iple        │  3.14   │
├─────────────┼─────────┤
│ utch        │  3.1132 │
├─────────────┼─────────┤
│ sembly      │  3.0786 │
├─────────────┼─────────┤
│ phrine      │  3.0376 │
├─────────────┼─────────┤
│ chance      │  3.0277 │
├─────────────┼─────────┤
│ forum       │  3.0182 │
├─────────────┼─────────┤
│ umni        │  3.0126 │
├─────────────┼─────────┤
│ Pacific     │  2.9834 │
├─────────────┼─────────┤
│ uders       │  2.9482 │
├───────────

In [46]:
feature_interpretation = get_response(llm_client, examples_clean_text, top_tokens)
print(feature_interpretation)

(Part 1)
Step 1.
ACTIVATING TOKENS: "in the county", "-,", " and", " than", "based", "Ab", ",, ", ")," "," "," ",",", " and".
PREVIOUS TOKENS: "relocated", "in", "Paul", "Zarrin", "galleries", "Strength:", "more", "prohibited".

Step 2.
The activating tokens are conjunctions, punctuation, or prepositions.
The previous tokens don't seem to have a clear pattern apart from being nouns or verbs.
The neuron seems to activate on specific grammatical elements rather than content tokens.

Step 3.
- The neuron activates on grammatical elements such as conjunctions, punctuation, and prepositions.
- The activating token sometimes follows a verb or a noun but there doesn't seem to be a clear pattern.

(Part 2)
Step 4.
SIMILAR TOKENS: Many of the top logits like "arth", "rers", "qua", " assian", "cember", " rer", "iple", "utch", "sembly", "phrine" seems to be fragments of words rather than full tokens. They seem to be related with letter "r".

Step 5.
[EXPLANATION]: This neuron seems to activate on

In [47]:
# Same thing
feature_idx = 1
example_html, examples_clean_text = display_top_k_activating_examples(model, feature_scores[:, feature_idx, :], owt_tokens_torch, k=15, display_html=False)
top_tokens, top_logits = get_top_k_tokens(model, sae, features[feature_idx], k=20, act_strength=5)
pretty_print_tokens_logits(top_tokens, top_logits)
feature_interpretation = get_response(llm_client, examples_clean_text, top_tokens)
print(feature_interpretation)

╒══════════════╤═════════╕
│ Token        │   Logit │
╞══════════════╪═════════╡
│ ividual      │  4.4617 │
├──────────────┼─────────┤
│  confir      │  4.3262 │
├──────────────┼─────────┤
│ legate       │  4.1775 │
├──────────────┼─────────┤
│ hots         │  3.8846 │
├──────────────┼─────────┤
│ breaker      │  3.8705 │
├──────────────┼─────────┤
│ uality       │  3.7917 │
├──────────────┼─────────┤
│  festivals   │  3.7612 │
├──────────────┼─────────┤
│ ulk          │  3.6968 │
├──────────────┼─────────┤
│ vertisements │  3.5496 │
├──────────────┼─────────┤
│ worker       │  3.5273 │
├──────────────┼─────────┤
│ lander       │  3.5042 │
├──────────────┼─────────┤
│ aily         │  3.4963 │
├──────────────┼─────────┤
│  arrivals    │  3.4198 │
├──────────────┼─────────┤
│  sugg        │  3.4051 │
├──────────────┼─────────┤
│ emp          │  3.3741 │
├──────────────┼─────────┤
│ queue        │  3.3683 │
├──────────────┼─────────┤
│ spawn        │  3.3635 │
├──────────────┼─────────┤
│

You can also pass in and boost logits for multiple features at a time.

In [51]:
top_tokens, top_logits = get_top_k_tokens(model, sae, features, k=10, act_strength=5)

pretty_print_tokens_logits(top_tokens, top_logits)


examples_html, examples_clean_text = display_top_k_activating_examples_sum(model, feature_scores, owt_tokens, [0, 1], k=5, show_score=True)

feature_interpretation = get_response(llm_client, examples_clean_text, top_tokens)
print(feature_interpretation)

╒══════════╤═════════╕
│ Token    │   Logit │
╞══════════╪═════════╡
│ hots     │  7.4445 │
├──────────┼─────────┤
│ legate   │  7.3243 │
├──────────┼─────────┤
│ hower    │  6.7189 │
├──────────┼─────────┤
│ umn      │  6.7184 │
├──────────┼─────────┤
│ ividual  │  6.7176 │
├──────────┼─────────┤
│  confir  │  6.5577 │
├──────────┼─────────┤
│ levision │  6.4419 │
├──────────┼─────────┤
│ rers     │  6.2546 │
├──────────┼─────────┤
│ utch     │  6.2519 │
├──────────┼─────────┤
│ pool     │  6.1928 │
╘══════════╧═════════╛


(Part 1)
Step 1.
ACTIVATING TOKENS: "Pal", "Dud", "Mr", "Ton", "Al", "p", "Darn", "Mold", "Dot", "Plum", "Put", "Dart", "Pl", "Rad", "Mud", "Plot", "L", "op", "Tan"
PREVIOUS TOKENS: "Mr", "&", "D", "u", "Nut", "the", "v", "Pl", "Art"

Step 2.
The activating tokens appear to be fragments or short abbreviations of words in a sentence or mathematical notation. These activated tokens often follow names or titles. The previous tokens are often names (e.g., "Mr") or letters associated with mathematical notation or abbreviation. These activating strings of tokens appear to be part of a larger pattern (e.g., chains of tokens together form a more meaningful phrase or concept).

Step 3.
- The examples contain names, acronyms, or abbreviations.
- The activating tokens often appear part of a larger whole, forming a meaningful concept when strung together. 
- Some examples involve mathematical or programming notation.

(Part 2)
Step 4.
SIMILAR TOKENS: "hots", "legate", "hower", "umn", "ividual", " 

Then, you can just pass it off to GPT-4 to interpret what's going on. Note that I haven't got access to `GPT-4o` with my credits yet, so this will have to wait a few days.

In [38]:
feature_interpretation = get_response(llm_client, examples_clean_text, top_tokens)

In [40]:
print(feature_interpretation)

(Part 1)
Step 1.
ACTIVATING TOKENS: "in the county", "days", "asia", "half, and", "ial,", "Byndom", "ia,", "plate", "to", "resett".
PREVIOUS TOKENS: "evacuated in", "-", "un", "par", "cliq", "Carr", "Pers", "name", "res", "charact".

Step 2.
The activating tokens are a mixture of prepositions, conjunctions, parts of words, days of the week and multipart words. 
The previous tokens have nothing in common.

Step 3.
- Many activating tokens are parts of words or phrases.
- The texts geographically widespread places.

(Part 2)
Step 4.
SIMILAR TOKENS: "arth", "rers", "rovers", "rer".
These tokens seem to be part of words, particularly endings part of nouns, adjectives or even verbs. 

Step 5:
[EXPLANATION]: Parts of words, notably the endings of nouns, verbs, or adjectives.


Finally, we can pass in multiple features at once to see the max activating examples for features together.

In [44]:
_ = display_top_k_activating_examples_sum(model, feature_scores, owt_tokens, [0, 1], k=5, show_score=True)

However, instead of passing in individual features for specific components in specific layers, I created an object called `CircuitPrediction` to basically store all this stuff for you. I'll quickly illustrate how to use it in conjunction with the above.

In [47]:
cp = get_circuit_prediction(task='ioi', N=20)

100%|██████████| 20/20 [00:43<00:00,  2.17s/it]


The main thing you'll want to do with this is get features from certain components to look at on a specific task. The features for each component are stored in the circuit hypergraph. For instance:

In [48]:
cp.circuit_hypergraph

{'L0_H0': {'freq': 0.0, 'features': []},
 'L0_H1': {'freq': 0.800000011920929,
  'features': [9715,
   1846,
   451,
   16579,
   -1,
   3949,
   10094,
   -1,
   23825,
   451,
   23825,
   17242,
   5142,
   13846,
   451,
   9715,
   17242,
   451,
   -1,
   17242,
   451,
   4229,
   17242,
   451,
   451,
   -1,
   14731,
   451,
   3501]},
 'L0_H2': {'freq': 0.0, 'features': []},
 'L0_H3': {'freq': 0.5,
  'features': [18802,
   2591,
   11470,
   11470,
   11470,
   21859,
   16579,
   11470,
   17242,
   11470,
   14731,
   17242]},
 'L0_H4': {'freq': 0.0, 'features': []},
 'L0_H5': {'freq': 0.8999999761581421,
  'features': [3392,
   16230,
   3545,
   9715,
   11883,
   10404,
   24085,
   20904,
   17083,
   18034,
   5690,
   4515,
   18034,
   16230,
   10404,
   14037,
   3160,
   21859,
   23663,
   18034,
   1412,
   16132,
   21859,
   3160,
   21339,
   3160,
   7161,
   21859,
   23603]},
 'L0_H6': {'freq': 0.949999988079071,
  'features': [-1,
   -1,
   -1,
   -1,
  

If you want to look at MLP 3, all you have to do is access it:

In [ ]:
cp.circuit_hypergraph['MLP3']

And just repeat what we did above:

In [ ]:
features = list(set(cp.circuit_hypergraph['MLP3']['features']))
transcoder = transcoders[3]
feature_scores = get_feature_scores(model, transcoder, owt_tokens_torch, features, batch_size=64)

In [ ]:
feature_idx = 0 # corresponding to 16513
example_html, examples_clean_text = display_top_k_activating_examples(model, feature_scores[:, 0, :], owt_tokens_torch, k=5, show_score=True)

There's a few other methods, but you probably don't need to bother with those.

In [ ]:
_ = cp.unique_feature_array(visualize=True)

# Playground

What do we want to actually look for?
* We could take specific components, and look at all their features across the circuit hypergraph, then get some sort of "mass autointerpretation" of what this feature is doing. I think for this you'd need to also feed in information from where it activates on the actual circuit. Might seem a bit soft and qualitative, but if you do it principled enough, it could be useful. Also try weighting the cluster max-act examples + logits by how often the feature shows up.
* Look at what features co-occur together in examples. Should give more signal than just looking at features that activate heaps. (Also look at features that activate strongly across all examples as well though.) 

## Feature cluster interpretation of model components

## Co-occurrence of features

In [1]:
from autointerpretability import *

cp = get_circuit_prediction(task='ioi', N=2)

/Users/charlesoneill/miniconda3/envs/anu/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded pretrained model gpt2-small into HookedTransformer
Loaded pretrained model gpt2-small into HookedTransformer

Loading SAEs...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
100%|██████████| 12/12 [00:07<00:00,  1.60it/s]



Loading Transcoders...


100%|██████████| 2/2 [00:05<00:00,  2.85s/it]


In [4]:
# Go through co-occurrence dict and print out the non-empty sets
for k, v in cp.co_occurrence_dict.items():
    for kk, vv in v.items():
        if vv:
            print(k, kk, vv)

('attn_head', 0, 1) ('attn_head', 0, 6) [(451, -1), (-1, 451), (-1, 451), (9715, -1), (451, -1)]
('attn_head', 0, 1) ('mlp_feature', 0) [(451, 17305), (17305, 451), (20546, 451), (17305, 451), (20546, 451), (10461, 451), (17305, 451), (20546, 451), (10461, 451), (13881, 451), (17305, 451), (20546, 451), (10461, 451), (13881, 451), (9715, 17305), (9715, 20546), (9715, 10461), (9715, 13881), (451, 17305), (451, 20546), (451, 10461), (451, 13881)]
('attn_head', 0, 1) ('mlp_feature', 4) [(22867, 451), (9715, 22867), (451, 22867)]
('attn_head', 0, 1) ('mlp_feature', 5) [(6307, 451), (9715, 6307), (451, 6307)]
('attn_head', 0, 1) ('attn_head', 6, 1) [(451, 18591), (9715, 18591), (451, 18591)]
('attn_head', 0, 1) ('attn_head', 7, 3) [(451, -1), (9715, -1), (451, -1)]
('attn_head', 0, 1) ('mlp_feature', 7) [(451, 16407), (9715, 16407), (451, 16407)]
('attn_head', 0, 1) ('attn_head', 9, 11) [(451, -1), (9715, -1), (451, -1)]
('attn_head', 0, 1) ('attn_head', 10, 7) [(451, -1), (9715, -1), (451,

In [11]:
# Clean up co-occurrence dict
cooc_dict = cp.co_occurrence_dict.copy()

# Start by removing all tuples with -1 as one of the pairs
for k, v in cooc_dict.items():
    for i, (kk, vv) in enumerate(v.copy().items()):
        if vv:
            for i, feat_tuple in enumerate(vv):
                if feat_tuple[0] == -1 or feat_tuple[1] == -1:
                    del cooc_dict[k][kk][i]
        else:
            # Remove empty sets
            del cooc_dict[k][kk]
                

# for k, v in cooc_dict.items():
#     for kk, vv in v.items():
#         if vv:
#             print(k, kk, vv)

In [12]:
cooc_dict

{('attn_head', 0, 0): {},
 ('attn_head',
  0,
  1): {('mlp_feature', 0): [(451, 17305),
   (17305, 451),
   (20546, 451),
   (17305, 451),
   (20546, 451),
   (10461, 451),
   (17305, 451),
   (20546, 451),
   (10461, 451),
   (13881, 451),
   (17305, 451),
   (20546, 451),
   (10461, 451),
   (13881, 451),
   (9715, 17305),
   (9715, 20546),
   (9715, 10461),
   (9715, 13881),
   (451, 17305),
   (451, 20546),
   (451, 10461),
   (451, 13881)], ('mlp_feature', 4): [(22867, 451),
   (9715, 22867),
   (451, 22867)], ('mlp_feature', 5): [(6307, 451),
   (9715, 6307),
   (451, 6307)], ('attn_head', 6, 1): [(451, 18591),
   (9715, 18591),
   (451, 18591)], ('mlp_feature', 7): [(451, 16407),
   (9715, 16407),
   (451, 16407)], ('attn_head', 11, 1): [(14155, 451),
   (9715, 14155),
   (451, 14155),
   (14155, 451),
   (14155, 9715)]},
 ('attn_head', 0, 2): {},
 ('attn_head', 0, 3): {},
 ('attn_head', 0, 4): {},
 ('attn_head', 0, 5): {},
 ('attn_head', 0, 6): {('mlp_feature', 0): []},
 ('attn

In [13]:
def clean_co_occurrence_dict(co_occurrence_dict):
    # Create a new dictionary to hold the cleaned data
    cleaned_dict = {}

    for component, co_occurrences in co_occurrence_dict.items():
        # Create a new sub-dictionary for the current component
        cleaned_sub_dict = {}

        for other_component, feature_tuples in co_occurrences.items():
            # Filter out tuples containing -1
            filtered_tuples = [t for t in feature_tuples if -1 not in t]

            if filtered_tuples:  # Only add if there are remaining tuples
                cleaned_sub_dict[other_component] = filtered_tuples

        if cleaned_sub_dict:  # Only add if there are remaining entries
            cleaned_dict[component] = cleaned_sub_dict

    return cleaned_dict

# Example usage with the provided co_occurrence_dict
cleaned_co_occurrence_dict = clean_co_occurrence_dict(cp.co_occurrence_dict)

# Print the cleaned dictionary
for k, v in cleaned_co_occurrence_dict.items():
    for kk, vv in v.items():
        print(k, kk, vv)

('attn_head', 0, 1) ('mlp_feature', 0) [(451, 17305), (17305, 451), (20546, 451), (17305, 451), (20546, 451), (10461, 451), (17305, 451), (20546, 451), (10461, 451), (13881, 451), (17305, 451), (20546, 451), (10461, 451), (13881, 451), (9715, 17305), (9715, 20546), (9715, 10461), (9715, 13881), (451, 17305), (451, 20546), (451, 10461), (451, 13881)]
('attn_head', 0, 1) ('mlp_feature', 4) [(22867, 451), (9715, 22867), (451, 22867)]
('attn_head', 0, 1) ('mlp_feature', 5) [(6307, 451), (9715, 6307), (451, 6307)]
('attn_head', 0, 1) ('attn_head', 6, 1) [(451, 18591), (9715, 18591), (451, 18591)]
('attn_head', 0, 1) ('mlp_feature', 7) [(451, 16407), (9715, 16407), (451, 16407)]
('attn_head', 0, 1) ('attn_head', 11, 1) [(14155, 451), (9715, 14155), (451, 14155), (14155, 451), (14155, 9715)]
('mlp_feature', 0) ('attn_head', 0, 1) [(451, 17305), (17305, 451), (20546, 451), (17305, 451), (20546, 451), (10461, 451), (17305, 451), (20546, 451), (10461, 451), (13881, 451), (17305, 451), (20546, 45